In [1]:
import pandas as pd
pd.options.display.max_colwidth = 100
import os
import pandas as pd
import numpy as np
import SimpleITK as sitk
import numpy as np
import radiomics
from radiomics import featureextractor
from radiomics import firstorder, shape
from radiomics import featureextractor, firstorder
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import torch
from typing import Tuple
import nibabel as nib
import warnings
warnings.filterwarnings('ignore')


In [2]:
#edema: 1
#necrosis: 3
#enhancing tumor: 2

In [3]:
def maximum3DDiameter(image,mask):
    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False


    # Define extractor object
    extractor = featureextractor.RadiomicsFeatureExtractor(label = 1, **settings)

    # Enable features from the shape category
    extractor.enableFeatureClassByName('shape')

    # Compute features
    featureVector = extractor.execute(image, mask)

    # Extract the Max 3D Diameter feature value
    max_3d_diameter = featureVector['original_shape_Maximum3DDiameter']
    x = max_3d_diameter.astype(float)

    return x



def interquartileRange(image, mask):
    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the first order category
    extractor.enableFeatureClassByName('firstorder')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label=5)

    # Extract the Interquartile Range feature value
    x = featureVector['original_firstorder_InterquartileRange']

    x = x.astype(float)

    return x


def meanAbsoluteDeviation(image, mask):
    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    # Define extractor object
    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the first order category
    extractor.enableFeatureClassByName('firstorder')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label=5)

    # Extract the Interquartile Range feature value
    x = featureVector['original_firstorder_MeanAbsoluteDeviation']

    x = x.astype(float)

    return x

def my_mean(image, mask):
    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the first order category
    extractor.enableFeatureClassByName('firstorder')

    # Compute features
    featureVector = extractor.execute(image, ma_merged)

    # Extract the Interquartile Range feature value
    x = featureVector['original_firstorder_Mean']

    x = x.astype(float)

    return x

# def median_whole(image, mask):
#     # Define settings for feature extraction
#     settings = {}
#     settings['binWidth'] = 5
#     settings['resampledPixelSpacing'] = None
#     settings['normalize'] = False

#     image =sitk.ReadImage(image)
#     mask =sitk.ReadImage(mask)

#     # Define extractor object
#     extractor = featureextractor.RadiomicsFeatureExtractor(label = 1&2&3, **settings)

#     # Enable features from the first order category
#     extractor.enableFeatureClassByName('firstorder')

#     # Compute features
#     featureVector = extractor.execute(image, mask)

#     # Extract the Interquartile Range feature value
#     x = featureVector['original_firstorder_Median']

#     x = x.astype(float)

#     return x

def median_whole(image, mask):
    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (1, 2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the first order category
    extractor.enableFeatureClassByName('firstorder')

    # Compute features for label 1, 2, and 3
    featureVector = extractor.execute(image, ma_merged, label=5)

    # Extract the Interquartile Range feature value
    x = featureVector['original_firstorder_Median']

    x = x.astype(float)

    return x


def median_core(image, mask):
    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)
    # Define extractor object
    extractor = featureextractor.RadiomicsFeatureExtractor(label = 2&3, **settings)

    # Enable features from the first order category
    extractor.enableFeatureClassByName('firstorder')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Interquartile Range feature value
    x = featureVector['original_firstorder_Median']

    x = x.astype(float)

    return x

def get_variance(image, mask):
    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False


    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the first order category
    extractor.enableFeatureClassByName('firstorder')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Interquartile Range feature value
    x = featureVector['original_firstorder_Variance']

    x = x.astype(float)

    return x

def ten_Percentile(image, mask):
    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the first order category
    extractor.enableFeatureClassByName('firstorder')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Interquartile Range feature value
    x = featureVector['original_firstorder_10Percentile']

    x = x.astype(float)

    return x

def grayLevelNonUniformityNormalized (image,mask):

    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    

    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (1, 2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the shape category
    extractor.enableFeatureClassByName('glrlm')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Max 3D Diameter feature value
    max_3d_diameter = featureVector['original_glrlm_GrayLevelNonUniformityNormalized']
    x = max_3d_diameter.astype(float)

    return x

def grayLevelVariance (image,mask):

    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (1, 2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the shape category
    extractor.enableFeatureClassByName('glrlm')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Max 3D Diameter feature value
    max_3d_diameter = featureVector['original_glrlm_GrayLevelVariance']
    x = max_3d_diameter.astype(float)

    return x

def smallAreaEmphasis (image,mask):

    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (1, 2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the shape category
    extractor.enableFeatureClassByName('glszm')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Max 3D Diameter feature value
    max_3d_diameter = featureVector['original_glszm_SmallAreaEmphasis']
    x = max_3d_diameter.astype(float)

    return x

def smallAreaHighGraylevelEmphasis (image,mask):

    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

   
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (1, 2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the shape category
    extractor.enableFeatureClassByName('glszm')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Max 3D Diameter feature value
    max_3d_diameter = featureVector['original_glszm_SmallAreaHighGrayLevelEmphasis']
    x = max_3d_diameter.astype(float)

    return x


def smallAreaLowGrayLevelEmphasis (image,mask):

    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

   
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (1, 2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the shape category
    extractor.enableFeatureClassByName('glszm')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Max 3D Diameter feature value
    max_3d_diameter = featureVector['original_glszm_SmallAreaLowGrayLevelEmphasis']
    x = max_3d_diameter.astype(float)

    return x

def smallAreaHighGrayLevelEmphasis (image,mask):

    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

   
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (1, 2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the shape category
    extractor.enableFeatureClassByName('glszm')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Max 3D Diameter feature value
    max_3d_diameter = featureVector['original_glszm_SmallAreaHighGrayLevelEmphasis']
    x = max_3d_diameter.astype(float)

    return x


def busyness (image,mask):

    # Define settings for feature extraction
    settings = {}
    settings['binWidth'] = 5
    settings['resampledPixelSpacing'] = None
    settings['normalize'] = False

    
    ma_arr = sitk.GetArrayFromImage(mask)
    

    # Define extractor object
    for l in (1, 2, 3):
        ma_arr[ma_arr == l] = 5

    ma_merged = sitk.GetImageFromArray(ma_arr)
    ma_merged.CopyInformation(mask)  # geometric information

    extractor = featureextractor.RadiomicsFeatureExtractor(label = 5, **settings)

    # Enable features from the shape category
    extractor.enableFeatureClassByName('ngtdm')

    # Compute features
    featureVector = extractor.execute(image, ma_merged, label = 5)

    # Extract the Max 3D Diameter feature value
    max_3d_diameter = featureVector['original_ngtdm_Busyness']
    x = max_3d_diameter.astype(float)

    return x



In [4]:
T1c_filename = "T1wCE/T1CE_to_SRI.nii.gz"
T1_filename = "T1w/T1_to_SRI.nii.gz"
T2_filename = "T2w/T2_to_SRI.nii.gz"
fl_filename = "FLAIR/FL_to_SRI.nii.gz"
mask_suffix = "/home/rfyahya/Desktop/Rawan/brain/clean/segmented_real_ids/train/BRATS_"

rootdir_onlysurv = "/home/rfyahya/Desktop/Rawan/brain/clean/preprocessed/train/"

osinfo = pd.read_csv("/home/rfyahya/Desktop/Rawan/brain/clean/Multi-View-MRI-Approach-for-Classification-of-MGMT-Methylation-in-Glioblastoma-Patients/dataframes/training_set_t2w.csv")

subjlist = list(osinfo["BraTS21ID"])
subjlist = [str(i).zfill(5) for i in subjlist]

overallfeatdf = pd.DataFrame(data=[],
                             columns= ['Maximum_3D_Diameter', 
                                            'Interquartile_Range', 
                                            'Mean_Absolute_Deviation', 
                                            'Mean', 'Median_Core', 
                                            'Median_Whole', 'Variance', 
                                            'tenPercentile', 
                                            'Graylevel_non_uniformity_normalized', 
                                            'Graylevel_variance', 
                                            'Small_area_emphasis', 
                                            'Small_area_high_graylevel_emphasis_flair', 
                                            'Small_area_low_graylevel_emphasis', 
                                            'Busyness', 
                                            'Small_area_high_graylevel_emphasis_t2', 'ID'])

overallfeatdf.index.name = "ID"

Maximum_3D_Diameter = []
Interquartile_Range = []
Mean_Absolute_Deviation= []
Mean = []
Median_Core = []
Median_Whole = []
Variance = []
tenPercentile = []
Graylevel_non_uniformity_normalized =[]
Graylevel_variance= []
Small_area_emphasis = []
Small_area_high_graylevel_emphasis_flair = []
Small_area_low_graylevel_emphasis = []
Busyness = []
Small_area_high_graylevel_emphasis_t2 = []
ids = []

for subj in tqdm(subjlist):
   
    t1wce_subdir = os.path.join(rootdir_onlysurv+subj+'/T1wCE/T1CE_to_SRI.nii.gz')
    #t1_subdir = os.path.join(rootdir_onlysurv+subj+'/T1w/T1_to_SRI.nii.gz')
    t2_subdir = os.path.join(rootdir_onlysurv+subj+'/T2w/T2_to_SRI.nii.gz')
    flair_subdir = os.path.join(rootdir_onlysurv+subj+'/FLAIR/FL_to_SRI.nii.gz')
    
    p = int(subj)
    p2= str("{:04d}".format(p))
    maskpath = os.path.join(mask_suffix+p2+'.nii.gz')


    t1wce_image =sitk.ReadImage(t1wce_subdir)
    #t1_image =sitk.ReadImage(t1_subdir)
    t2_image =sitk.ReadImage(t2_subdir)
    flair_image =sitk.ReadImage(flair_subdir)
    mask =sitk.ReadImage(maskpath)

    max_diameter = maximum3DDiameter(t1wce_image, mask)
    inter_range = interquartileRange(t1wce_image, mask)
    mean_abs_dev = meanAbsoluteDeviation(t1wce_image, mask)
    mean = my_mean(t1wce_image, mask)
    median_whole_tumor = median_whole(t1wce_image, mask)
    median_core_tumor = median_core(t1wce_image, mask)
    variance = get_variance(t1wce_image, mask)
    
    ten_percentile = ten_Percentile(flair_image, mask)

    gl_non_uniformity = grayLevelNonUniformityNormalized(flair_image, mask)
    gl_variance = grayLevelVariance(flair_image, mask)
    small_area_emphasis = smallAreaEmphasis(flair_image, mask)
    small_area_high_gl_emphasis_flair = smallAreaHighGraylevelEmphasis(flair_image, mask)
    small_area_low_gl_emphasis = smallAreaLowGrayLevelEmphasis(flair_image, mask)
    busyness_value = busyness(flair_image, mask)
    small_area_high_gl_emphasis_t2 = smallAreaHighGraylevelEmphasis(t2_image, mask)


        
    Maximum_3D_Diameter.append(max_diameter)
    Interquartile_Range.append(inter_range)
    Mean_Absolute_Deviation.append(mean_abs_dev)
    Mean.append(mean)
    Median_Core.append(median_core_tumor)
    Median_Whole.append(median_whole_tumor)
    Variance.append(variance)
    tenPercentile.append(ten_percentile)
    Graylevel_non_uniformity_normalized.append(gl_non_uniformity)
    Graylevel_variance.append(gl_variance)
    Small_area_emphasis.append(small_area_emphasis)
    Small_area_high_graylevel_emphasis_flair.append(small_area_high_gl_emphasis_flair)
    Small_area_low_graylevel_emphasis.append(small_area_low_gl_emphasis)
    Busyness.append(busyness_value)
    Small_area_high_graylevel_emphasis_t2.append(small_area_high_gl_emphasis_t2)
    ids.append(subj)
    
    currfeaturedf = pd.DataFrame(data=[[Maximum_3D_Diameter, Interquartile_Range, Mean_Absolute_Deviation, 
                                        Mean, Median_Core, Median_Whole, Variance, tenPercentile, 
                                        Graylevel_non_uniformity_normalized, Graylevel_variance, 
                                        Small_area_emphasis, Small_area_high_graylevel_emphasis_flair, 
                                        Small_area_low_graylevel_emphasis, Busyness, 
                                        Small_area_high_graylevel_emphasis_t2,ids]],

                                 columns= ['Maximum_3D_Diameter', 
                                            'Interquartile_Range', 
                                            'Mean_Absolute_Deviation', 
                                            'Mean', 'Median_Core', 
                                            'Median_Whole', 'Variance', 
                                            'tenPercentile', 
                                            'Graylevel_non_uniformity_normalized', 
                                            'Graylevel_variance', 
                                            'Small_area_emphasis', 
                                            'Small_area_high_graylevel_emphasis_flair', 
                                            'Small_area_low_graylevel_emphasis', 
                                            'Busyness', 
                                            'Small_area_high_graylevel_emphasis_t2', 'ID'])
    print(currfeaturedf)

    overallfeatdf = overallfeatdf.append(currfeaturedf.reset_index(drop=True), sort =False)
overallfeatdf.to_csv('Train_RAD.csv')

  0%|          | 0/420 [00:00<?, ?it/s]GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, 

    Maximum_3D_Diameter  Interquartile_Range Mean_Absolute_Deviation  \
0  [101.68579055108928]  [245.7269744873047]    [122.31314574556318]   

                  Mean         Median_Core         Median_Whole  \
0  [462.9341225333326]  [444.513427734375]  [286.5868225097656]   

               Variance         tenPercentile  \
0  [19659.821387408083]  [234.68841857910155]   

  Graylevel_non_uniformity_normalized   Graylevel_variance  \
0               [0.02600832988899388]  [129.8404353320693]   

    Small_area_emphasis Small_area_high_graylevel_emphasis_flair  \
0  [0.6480270869610024]                      [842.9129288772886]   

  Small_area_low_graylevel_emphasis             Busyness  \
0           [0.0008239297444666753]  [2.167747463779031]   

  Small_area_high_graylevel_emphasis_t2       ID  
0                   [2914.212838610235]  [00147]  


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

                        Maximum_3D_Diameter  \
0  [101.68579055108928, 103.20368210485515]   

                      Interquartile_Range  \
0  [245.7269744873047, 198.6139373779297]   

                    Mean_Absolute_Deviation  \
0  [122.31314574556318, 115.94398301655166]   

                                     Mean  \
0  [462.9341225333326, 725.1345282800922]   

                             Median_Core  \
0  [444.513427734375, 735.8919677734375]   

                             Median_Whole  \
0  [286.5868225097656, 410.2227478027344]   

                                 Variance  \
0  [19659.821387408083, 23804.3005344124]   

                          tenPercentile  \
0  [234.68841857910155, 304.6728515625]   

           Graylevel_non_uniformity_normalized  \
0  [0.02600832988899388, 0.021579079449600854]   

                        Graylevel_variance  \
0  [129.8404353320693, 242.17481393402204]   

                        Small_area_emphasis  \
0  [0.6480270869610024, 0.684

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

                                                                                   Maximum_3D_Diameter  \
0  [101.68579055108928, 103.20368210485515, 115.56383517346592, 56.542019772908716, 103.12128781197...   

                                                                                   Interquartile_Range  \
0  [245.7269744873047, 198.6139373779297, 1336.9541015625, 2308.529296875, 142.1927490234375, 250.0...   

                                                                               Mean_Absolute_Deviation  \
0  [122.31314574556318, 115.94398301655166, 774.7812355741405, 1155.816990164313, 87.3581708799427,...   

                                                                                                  Mean  \
0  [462.9341225333326, 725.1345282800922, 4982.898106677835, 3388.8752205055594, 366.95976193906324...   

                                                                                           Median_Core  \
0  [444.513427734375, 735.8919677734375, 4

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

                                                                                   Maximum_3D_Diameter  \
0  [101.68579055108928, 103.20368210485515, 115.56383517346592, 56.542019772908716, 103.12128781197...   

                                                                                   Interquartile_Range  \
0  [245.7269744873047, 198.6139373779297, 1336.9541015625, 2308.529296875, 142.1927490234375, 250.0...   

                                                                               Mean_Absolute_Deviation  \
0  [122.31314574556318, 115.94398301655166, 774.7812355741405, 1155.816990164313, 87.3581708799427,...   

                                                                                                  Mean  \
0  [462.9341225333326, 725.1345282800922, 4982.898106677835, 3388.8752205055594, 366.95976193906324...   

                                                                                           Median_Core  \
0  [444.513427734375, 735.8919677734375, 4

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [ ]:
It should be noted that the authors of the nn-unet for brats segmentation paper referred to the brain tumor sub-regions as edema, enhancing tumor, and non-enhancing tumor, which they later described as being potentially inconsistent and subjective. However, it is clear from the context of their paper that the non-enhancing tumor label corresponds to the same sub-region that we have segmented as necrosis. Additionally, the brats2020 dataset uses the label necrosis/non-enhancing tumor for this same sub-region, further supporting the interchangeable use of these terms. Therefore, the present experiment's segmentation of the sub-regions into enhancing tumor, necrosis, and edema can still be used to extract radiomics features following the approach proposed by the nn-unet for brats segmentation paper, with the replacement of the non-enhancing tumor label with the necrosis label, without significantly affecting the overall results of the study.

In [ ]:
for i in Busyness:
    print (float(i))

In [ ]:
for i in Skewness:
    print (float(i))

In [ ]:
mask = nib.load("/home/rfyahya/Desktop/Rawan/brain/clean/segmented_real_ids/train/BRATS_0147.nii.gz").get_fdata()
unique_vals = np.unique(mask)
print(unique_vals)

In [ ]:
counts = {}

# loop over each element in the 3D array
for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        for k in range(mask.shape[2]):
            # get the value of the current element
            elem = mask[i, j, k]
            
            # add the element to the dictionary if it's not already there
            if elem not in counts:
                counts[elem] = 1
            # increment the count for the element if it's already in the dictionary
            else:
                counts[elem] += 1

# print the resulting dictionary
print(counts)

In [ ]:
1